In [1]:
import logging
import pandas_gbq
from datetime import datetime
from web3 import Web3
from google.oauth2 import service_account

from uniswap_utils import getPoolPrice #https://github.com/amantay-a/misc/blob/master/uniswap_utils.py

token_dict = {'ETH':'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', #WETH
              'BTC':'0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', #WBTC
              'USDC':'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
               'DAI':'0x6B175474E89094C44Da98b954EedeAC495271d0F',
             '1INCH':'0x111111111117dC0aa78b770fA6A738034120C302',
              'AAVE':'0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9',
              'COMP':'0xc00e94Cb662C3520282E6f5717214004A7f26888',
               'DPI':'0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b',
               'FEI':'0x956F47F50A910163D8BF957Cf5846D573E7f87CA',
              'LINK':'0x514910771AF9Ca656af840dff83E8264EcF986CA',
               'SNX':'0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F',
               'UNI':'0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
               'YFI':'0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e',
              'LUNA':'0xd2877702675e6cEb975b4A1dFf9fb7BAF4C91ea9', #WLUNA
               'FTM':'0x4E15361FD6b4BB609Fa63C81A2be19d873717870',
             'SUSHI':'0x6B3595068778DD592e39A122f4f5a5cF09C90fE2',
               'CRV':'0xD533a949740bb3306d119CC777fa900bA034cd52',
               'CVX':'0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B',
               'LDO':'0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32',
             'STETH':'0xDFe66B14D37C77F4E9b180cEb433d1b164f0281D',
              'FRAX':'0x853d955aCEf822Db058eb8505911ED77F175b99e',
              'LUSD':'0x5f98805A4E8be255a32880FDeC7F6728C6568bA0',
              'SUSD':'0x57Ab1ec28D129707052df4dF418D58a2D46d5f51',
              'GUSD':'0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd',
              'USDT':'0xdAC17F958D2ee523a2206206994597C13D831ec7'
            }
# To be able to call historical prices, RPC must support archive mode, suggest using alchemy as it provide archive rpc for free
RPCEndpoint = 'https://eth-mainnet.alchemyapi.io/v2/FKfen82HuKwIHeV8wXZ5nPFCQ5r_VEt-'

w3 = Web3(Web3.HTTPProvider(RPCEndpoint, request_kwargs={'timeout': 20}))
print(f'Blockchain connected: {w3.isConnected()}')

Blockchain connected: True


In [2]:
def get_uni_price(w3, tokenA, tokenB, block, TWAPWindows, PoolFees):
    
    price_twap = getPoolPrice(w3, 
                              tokenA, 
                              tokenB,
                              TWAPWindows = TWAPWindows,
                              blocks = [block],
                              PoolFees = [3000, 10000] 
                             )
    
    #print(price_twap)
    ret = {}
    for w in TWAPWindows:
        s = 0
        cnt = 0
        for i, f in enumerate(PoolFees):
            price = price_twap[block][i]['twap'][w]
            #print('price', price)
            if price:
                s+= price
                cnt+=1
            #print('s=','s')    
        if cnt>0:        
            ret.update({'twap_'+str(w):s/cnt}) 
        else:
            ret.update({'twap_'+str(w):None})
    return ret

In [3]:
credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'

bq_select = '''with u as (select ticker, max(blockNumber) as twap_max_block from gearbox.uniswap_prices group by ticker) 
               select 
                   cl.ticker, cl.blockNumber, cl.price_decimal as price_chainlink
               from 
                   gearbox.oracle_price_history cl
                   left join u on u.ticker = cl.ticker
               where cl.base !='usd'
                   and cl.blockNumber > coalesce(twap_max_block,0)

            '''
df = pandas_gbq.read_gbq(bq_select, 
                             project_id=gcp_project_id,
                             progress_bar_type = None,)
display(df)


,ticker,blockNumber,price_chainlink
0,btc-eth,12069614.0,32.227933
1,btc-eth,12076097.0,32.012001
2,btc-eth,12082548.0,31.962009
3,btc-eth,12089035.0,32.070000
4,btc-eth,12095588.0,32.325852
...,...,...,...
57898,sushi-eth,14647968.0,0.001177
57899,sushi-eth,14648757.0,0.001153
57900,sushi-eth,14650746.0,0.001130
57901,sushi-eth,14652368.0,0.001104


In [4]:
df['tokenA'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[0].upper()])
df['tokenB'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[1].upper()])
df

,ticker,blockNumber,price_chainlink,tokenA,tokenB
0,btc-eth,12069614.0,32.227933,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
1,btc-eth,12076097.0,32.012001,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
2,btc-eth,12082548.0,31.962009,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
3,btc-eth,12089035.0,32.070000,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
4,btc-eth,12095588.0,32.325852,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
...,...,...,...,...,...
57898,sushi-eth,14647968.0,0.001177,0x6B3595068778DD592e39A122f4f5a5cF09C90fE2,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
57899,sushi-eth,14648757.0,0.001153,0x6B3595068778DD592e39A122f4f5a5cF09C90fE2,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
57900,sushi-eth,14650746.0,0.001130,0x6B3595068778DD592e39A122f4f5a5cF09C90fE2,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
57901,sushi-eth,14652368.0,0.001104,0x6B3595068778DD592e39A122f4f5a5cF09C90fE2,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


In [ ]:
twapwindows =  [1, 60*15, 60*20]
twapwindow_cols = ['twap_'+str(x) for x in twapwindows]
poolfee = [3000, 10000]
df[twapwindow_cols] = None
cnt = 0
print(datetime.utcnow(),'Start')
for x in df.itertuples():
    prices = get_uni_price(w3, x.tokenA, x.tokenB, int(x.blockNumber), twapwindows, poolfee)
    prices = list(prices.values())
    df.loc[df['blockNumber'] == x.blockNumber, twapwindow_cols] = prices
    cnt+=1
    if cnt%100==0:
        print(datetime.utcnow(),f'{cnt}/{len(df)}..')
display(df)

2022-04-25 10:13:16.835936 Start
2022-04-25 10:15:10.583969 100/57903..
2022-04-25 10:16:27.090641 200/57903..
2022-04-25 10:17:49.403626 300/57903..
2022-04-25 10:19:11.441542 400/57903..
2022-04-25 10:20:31.093012 500/57903..
2022-04-25 10:21:47.058238 600/57903..
2022-04-25 10:22:56.422562 700/57903..
2022-04-25 10:24:13.450181 800/57903..
2022-04-25 10:26:08.278943 900/57903..
2022-04-25 10:28:04.267844 1000/57903..
2022-04-25 10:29:59.237793 1100/57903..
2022-04-25 10:31:53.647433 1200/57903..
2022-04-25 10:33:50.012876 1300/57903..
2022-04-25 10:35:44.986051 1400/57903..
2022-04-25 10:37:35.312011 1500/57903..
2022-04-25 10:38:42.796778 1600/57903..
2022-04-25 10:39:51.024132 1700/57903..
2022-04-25 10:40:57.754768 1800/57903..
2022-04-25 10:42:08.149909 1900/57903..


In [111]:


get_uni_price(w3,
             '0x6B3595068778DD592e39A122f4f5a5cF09C90fE2',
             '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
              int(14652383.0),
              [1, 60*15, 60*20],
              [3000, 10000]
             )


{14652383: [{'pool': '0x73A6a761FE483bA19DeBb8f56aC5bbF14c0cdad1', 'PoolFee': 3000, 'twap': {1: 0.001128505980033411, 900: 0.001128505980033411, 1200: 0.001128505980033411}}, {'pool': '0x19E286157200418d6A1f7D1df834b82E65C920AA', 'PoolFee': 10000, 'twap': {1: 0.0011534916057825372, 900: 0.0011534916057825372, 1200: 0.0011534916057825372}}]}
price 0.001128505980033411
s= s
price 0.0011534916057825372
s= s
price 0.001128505980033411
s= s
price 0.0011534916057825372
s= s
price 0.001128505980033411
s= s
price 0.0011534916057825372
s= s


{'twap_1': None, 'twap_900': None, 'twap_1200': None}